### Example 1 :ConvNet

你所有的网络都来自nn.Module基类：
* 在构造函数中，声明你想使用的所有层。
* 在forward函数中，你可以定义模型从输入到输出将如何运行

In [10]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class MNISTConvNet(nn.Module):
    def __init__(self):
        # 这是你实例化所有模块的地方
        # 你可以稍后使用你在此给出的相同名称访问它们
        super(MNISTConvNet,self).__init__()
        self.conv1 = nn.Conv2d(1,10,5)
        self.pool1 = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(10,20,5)
        self.pool2 = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(320,50)
        self.fc2 = nn.Linear(50,10)
        
    #这是 forward 函数，它定义了我们在这里只接受一个输入的网络结构，
    #如果你远离，可以随意使用更多的输入。
    def forward(self,input):
        x = self.pool1(F.relu(self.conv1(input)))
        x = self.pool2(F.relu(self.conv2(x)))
        
        # 在你的模型定义中, 你可以疯狂地使用任意的python代码来定义你的模型结构,
        # 这些都是完全合法的, 并且会被autograd正确处理:
        # if x.gt(0) > x.numel() / 2:
        #      ...
        #
        # 你甚至可以做一个循环, 并重复使用相同的模块, 模块内部的模块不再
        # 处于临时状态, 所以你可以在 forward 时多次使用它们.
        # while x.norm(2) < 10:
        #    x = self.conv1(x)
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x
    
net = MNISTConvNet()
print(net)
        

MNISTConvNet (
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (pool1): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (pool2): MaxPool2d (size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (fc1): Linear (320 -> 50)
  (fc2): Linear (50 -> 10)
)


创建一个包含随机数据的单样本的mini-batch,并通过ConvNet发送样本。

In [12]:
input = Variable(torch.randn(1,1,28,28))
out = net(input)
print(out.size())

torch.Size([1, 10])


In [14]:
#定义一个虚拟目标标签，并使用损失函数计算error.
target = Variable(torch.LongTensor([3]))
loss_fn = nn.CrossEntropyLoss()
err = loss_fn(out,target)
err.backward()
print(err)

Variable containing:
 2.3789
[torch.FloatTensor of size 1]



ConvNet 的 out 是一个 Variable. 我们使用它来计算损失, 计算结果 err 也是一个 Variable. 在 err 上调用 .backward 将会通过 ConvNet 将梯度传播到它的权重.

In [16]:
print(net.conv1.weight)
print(net.conv1.weight.grad.size())
print(net.conv1.weight.data.norm())  # norm of the weight
print(net.conv1.weight.grad.data.norm())  # norm of the gradients

Parameter containing:
(0 ,0 ,.,.) = 
 -0.1764  0.0057  0.0445  0.0061  0.0376
  0.0031  0.1113 -0.1774 -0.1272  0.0410
 -0.0143  0.0059  0.0486 -0.1938  0.0335
 -0.1076  0.0296 -0.0670 -0.1987  0.0958
 -0.1837  0.1265 -0.1571 -0.1050 -0.1847

(1 ,0 ,.,.) = 
  0.0194 -0.1014 -0.1065 -0.0596  0.0985
 -0.1978  0.0089  0.0193 -0.1824 -0.1141
  0.1127 -0.1162  0.0454 -0.1359  0.1376
  0.0678 -0.0188  0.1989 -0.0285  0.0114
  0.0099 -0.0934  0.0677  0.1837 -0.1324

(2 ,0 ,.,.) = 
 -0.0025  0.0996  0.0791 -0.0920 -0.1033
  0.0933  0.1085  0.1291  0.0432  0.1915
 -0.0939 -0.1067 -0.1400  0.1442  0.0277
 -0.1524  0.0396 -0.0022  0.1047  0.0804
 -0.0738  0.1718  0.0652  0.0419  0.1983

(3 ,0 ,.,.) = 
  0.1867 -0.0591  0.1298  0.0287  0.0338
 -0.0561  0.0323 -0.0950  0.0922  0.0471
  0.1360  0.1524 -0.0913  0.1020  0.0365
  0.0740  0.0190 -0.1210 -0.0914 -0.0836
  0.1328  0.0521  0.0383 -0.0060  0.0320

(4 ,0 ,.,.) = 
  0.1761  0.0099  0.1225 -0.1637  0.1168
  0.0729  0.0948 -0.1005  0.1350  0.18

### Forward and Backward Function Hooks

你可以在一个 Module 或一个 Variable 上注册一个函数. hook 可以是 forward hook 也可以是一个 backward hook. 当 forward 被执行时 forward hook 将会被执行. backward hook 将在 backward 阶段被执行.

In [17]:
#forward_hook

def printnorm(self,input,output):
    #输入是打包输入的tuple
    #输出是一个 Variable。 output.data 是我们感兴趣的Tensor
    print('Inside' + self.__class__.__name__ + 'forward')
    print('')
    print('input : ' + type(input))
    print('input[0]: ', type(input[0]))
    print('output: ', type(output))
    print('')
    print('input size:', input[0].size())
    print('output size:', output.data.size())
    print('output norm:', output.data.norm())

    
net.conv2.register_forward_hook(printnorm)

out = net(input)  
    
  

In [ ]:
def printgradnorm(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('Inside class:' + self.__class__.__name__)
    print('')
    print('grad_input: ', type(grad_input))
    print('grad_input[0]: ', type(grad_input[0]))
    print('grad_output: ', type(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print('')
    print('grad_input size:', grad_input[0].size())
    print('grad_output size:', grad_output[0].size())
    print('grad_input norm:', grad_input[0].data.norm())


net.conv2.register_backward_hook(printgradnorm)

out = net(input)
err = loss_fn(out, target)
err.backward()    